## 1. Get five letter words

In [27]:
!pip install english-words
import english_words
word_list = english_words.english_words_lower_alpha_set

word_length = 5
flw = set([word for word in word_list if 
           (len(word) == word_length) &
           (word[0] == word[0].lower()) for obj in word_list])
flw.remove('u.s.a')
n_words = len(flw)

## 2. Define functions for winnowing down word list

In [2]:
# define function to get similarity

def similarity(base_word, guess):
  """
    assumes:
    1. suppose a letter is repeated. we can have it green in
       one spot and yellow in another.
  """
  similarity_dict = {}
  similarity_dict['matches'] = {}
  similarity_dict['others'] = []
  similarity_dict['not_in'] = []

  for i in range( len(base_word) ):

    # green square
    matched = False
    if base_word[i] == guess[i]:
      similarity_dict['matches'][i] = base_word[i]
      matched = True

    # yellow square or black square
    base_word_other = base_word[:i] + base_word[:(i + 1)]
    base_word_other = set(base_word_other)
    if guess[i] in base_word_other and not matched:
      similarity_dict['others'].append(guess[i])
    else:
      similarity_dict['not_in'].append(guess[i])

  return similarity_dict

# test
similarity('tiger', 'plane')

{'matches': {}, 'not_in': ['p', 'l', 'a', 'n'], 'others': ['e']}

In [3]:
# define function to see if a word is OK
# based on ONE similarity dict

def accept_singular(word, similarity_dict):
  """
    function to determine if a word is "knocked out" based on what we know.
  """
  matches = similarity_dict['matches']
  unmatched = list(word)
  for key in matches:
    if word[key] != matches[key]:
      return False
    else:
      unmatched.remove(matches[key])

  for ch in similarity_dict['not_in']:
    if ch in unmatched:
      return False

  for ch in similarity_dict['others']:
    if ch not in unmatched:
      return False
    else:
      unmatched.remove(ch)

  return True

def accept(word, similarity_dicts):
  """
    wrapper for accept_singular
  """
  for d in similarity_dicts:
    if not accept_singular(word, d):
      return False
  return True

sim = similarity('tiger', 'plane')
accept('tiser', [sim])

True

In [4]:
# function to take the dictionaries and narrow down the possible words remaining
def dicts_to_possible_words(dicts, possible_words):
  """
    with the dictionary of information given,
    kill the words that are no longer possible.
  """
  words_still_possible = possible_words.copy()
  words_to_remove = []
  for word in words_still_possible:
    if not accept(word, dicts):
      words_to_remove.append(word)

  for word in words_to_remove:
    words_still_possible.remove(word)
  return words_still_possible

dicts = [sim]
possible_words = flw
new_possible = dicts_to_possible_words(dicts, possible_words)

In [5]:
dicts = [
 
]

## Greedy algorithm to guess first word
(running time is currently 0.5 days)

In [20]:
max_cut = 9000 ** 3 # higher than any possible score
best_word = ''
scores = {}
c = 0
for base_word in flw:
  remaining_dict = flw - set((base_word, ))
  s = 0
  print('base word: {a}; best word so far: {b}; words completed: {c}; words remaining: {d}'.format(a=base_word, b=best_word, c=c, d=(len(flw) - c)))
  
  # calculate average narrowing down
  for guess in remaining_dict:
    sim = [similarity(base_word, guess)]
    rem = dicts_to_possible_words(sim, flw)
    s += len(rem)

  # replace the best word, if this is the best word
  if s < max_cut:
    best_word = base_word
    max_cut = s

  # store in dictionary
  scores[base_word] = s
  c += 1

best_word
  

base word: scoop; best word so far: ; words completed: 0; words remaining: 3213
base word: weiss; best word so far: scoop; words completed: 1; words remaining: 3212
base word: denny; best word so far: scoop; words completed: 2; words remaining: 3211
base word: worst; best word so far: denny; words completed: 3; words remaining: 3210
base word: buteo; best word so far: denny; words completed: 4; words remaining: 3209
base word: whiff; best word so far: buteo; words completed: 5; words remaining: 3208
base word: olden; best word so far: buteo; words completed: 6; words remaining: 3207
base word: dross; best word so far: olden; words completed: 7; words remaining: 3206
base word: hymen; best word so far: olden; words completed: 8; words remaining: 3205
base word: plume; best word so far: olden; words completed: 9; words remaining: 3204
base word: bunny; best word so far: olden; words completed: 10; words remaining: 3203
base word: dirty; best word so far: olden; words completed: 11; words

KeyboardInterrupt: ignored

In [ ]:
# sort all scores
import pandas as pd
score_df = pd.DataFrame(scores, index=['Score']).T
score_df.sort_values('Score')

## Vectorized calculation: functions

In [76]:
import numpy as np
l2n = {}
alphabet = 'abcdefghijklmnopqrstuvwxyz'
for i in range(len(alphabet)):
  l2n[alphabet[i]] = i

def word_to_vec(word):
  """
    Gives an appropriate vector representation of a word
    to be used in vectorized calculations
  """
  g = np.zeros( (len(word) + 2) * 26 ).astype(bool)
  for i in range(len(word)):
    base = i * 26
    add  = l2n[ word[i] ]
    g[ base + add ] = 1
    g[len(word) * 26 + add ] = 1
  g[-26:] = 1 - g[-52:-26]
  return g

def info(guess, target):
  n = 6 * 26
  g = np.zeros( 7 * 26 ).astype(int)
  g[:n] = guess[:n] * target[:n]
  g[n:] = np.maximum( target[n:] - guess[n:] , 0)
  return g

## Vectorized calculation: build word matrix

In [80]:
flw_list = list(flw)
word_matrix = np.array( [word_to_vec(word).reshape(7, 26) for word in flw_list] )
word_matrix[0:1, :]

SyntaxError: ignored

## Vectorized calculation:

In [ ]:
m            = word_matrix.shape[0]
words_killed = 0 # avg number killed
best_word.   = ''
scores.      = {}

for i in range( m ):

  # 0 Word we are guessing; word and vectorized form
  base_word = flw_list[i]
  guess = word_matrix[i:(i + 1), :, :]

  # 1 If we guess base_word, what is the "info content" given
  #   every potential target word?
  #   1.1 set up matrix
  info_matrix = np.zeros( word_matrix.shape ).astype(bool)
  #   1.2 match known letters (position and inclusion)
  info_matrix[:, :6, :] = ( guess[:, :6, :] & word_matrix[:, :6, :] )

  #   1.3 find letters in the guess that are not present in the other word
  info_matrix[:, 6, :] =  np.logical_not(word_matrix[:, 5, :]) & guess[:, 5, :]

  # 2 find what words we have killed given any target word
  n_words = 0
  for j in range( 7, m ): 
    rem_words = np.zeros_like(info_matrix).astype(bool)

    # 2.1 info we would get if this was the target word:
    info_unit  = info_matrix[j:(j + 1), :, :]

    # 2.1.1 assure we don't have a true somewhere in the info's
    #       known letters, but a false in that word
    rem_words[:, :6, :] = np.logical_not(
          info_unit[:, :6, :] & np.logical_not( word_matrix[:, :6, :] )
          )
    
    # 2.1.2 assure we don't have a letter that we know is _not_ present given
    #       info, but which _is_ present in the target word
    rem_words[:, 6, :] = np.logical_not( info_unit[:, 6, :] & word_matrix[:, 5, :] )
    count = rem_words.min(axis=1).min(axis=1)
    count = np.logical_not( count )
    n_words += count.sum()


  n_words = n_words / m
  if n_words > words_killed:
    words_killed = n_words
    best_word = flw_list[i]
  scores[ base_word ] = n_words 
  
  print('base word: {a}; best word so far: {b}; words completed: {c}; words remaining: {d}, killed: {e}'.format(a=base_word, 
                                                                                                   b=best_word, 
                                                                                                   c=i,
                                                                                                   d=(len(flw_list) - i),
                                                                                                   e=n_words)
  )


base word: scoop; best word so far: scoop; words completed: 0; words remaining: 3212, killed: 2681.5610211706103
base word: weiss; best word so far: weiss; words completed: 1; words remaining: 3211, killed: 2786.097135740971
base word: denny; best word so far: weiss; words completed: 2; words remaining: 3210, killed: 2756.7826899128268
base word: worst; best word so far: worst; words completed: 3; words remaining: 3209, killed: 2931.3032378580324
base word: buteo; best word so far: buteo; words completed: 4; words remaining: 3208, killed: 2946.195205479452
base word: whiff; best word so far: buteo; words completed: 5; words remaining: 3207, killed: 2226.5977584059774
base word: olden; best word so far: olden; words completed: 6; words remaining: 3206, killed: 2993.152864259029
base word: dross; best word so far: olden; words completed: 7; words remaining: 3205, killed: 2827.9380448318802
base word: hymen; best word so far: olden; words completed: 8; words remaining: 3204, killed: 2890.